## import library

In [10]:
!pip install tqdm
!pip install opencv-python
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset
import cv2
import pandas as pd 
import os
import random
import gc

%matplotlib inline


In [11]:
import os, cv2, itertools # cv2 -- OpenCV
import numpy as np 
import pandas as pd 
 
import matplotlib.pyplot as plt 
%matplotlib inline

# all function name

## 1. Sigmoid function(z)
## 2. initialize_parameters:
    initialiaze_with_zeros(dim)
## 3.propogate(w,b,X,Y)
##  4.optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False)
## 5.pedict(w,b,X)
## 6.model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False)

## 2 - Overview of the Problem set ##

**Problem Statement**: You are given a dataset ("data.h5") containing:
    - a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
    - a test set of m_test images labeled as cat or non-cat
    - each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

You will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.

Let's get more familiar with the dataset. Load the data by running the following code.

In [2]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

OSError: Unable to open file (unable to open file: name = 'datasets/train_catvnoncat.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [18]:
X_train, y_train = prep_data(train_images)
X_test, test_idx = prep_data(test_images)

NameError: name 'train_images' is not defined

In [17]:
len(X_train)

NameError: name 'X_train' is not defined

## Loading data

In [1]:
import PIL
print('Pillow Version:', PIL.__version__)

Pillow Version: 7.0.0


In [2]:
from PIL import Image

In [3]:
%matplotlib inline
 
import numpy as np
import matplotlib.pyplot as plt
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Algrithm

## 3 - General Architecture of the learning algorithm ##

It's time to design a simple algorithm to distinguish cat images from non-cat images.

You will build a Logistic Regression, using a Neural Network mindset. The following Figure explains why **Logistic Regression is actually a very simple Neural Network!**

<img src="images/LogReg_kiank.png" style="width:650px;height:400px;">

**Mathematical expression of the algorithm**:

For one example $x^{(i)}$:
$$z^{(i)} = w^T x^{(i)} + b \tag{1}$$
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})\tag{2}$$ 
$$ \mathcal{L}(a^{(i)}, y^{(i)}) =  - y^{(i)}  \log(a^{(i)}) - (1-y^{(i)} )  \log(1-a^{(i)})\tag{3}$$

The cost is then computed by summing over all training examples:
$$ J = \frac{1}{m} \sum_{i=1}^m \mathcal{L}(a^{(i)}, y^{(i)})\tag{6}$$

**Key steps**:
In this exercise, you will carry out the following steps: 
    - Initialize the parameters of the model
    - Learn the parameters for the model by minimizing the cost  
    - Use the learned parameters to make predictions (on the test set)
    - Analyse the results and conclude

### sigmoid function

## 4 - Building the parts of our algorithm ## 

The main steps for building a Neural Network are:
1. Define the model structure (such as number of input features) 
2. Initialize the model's parameters
3. Loop:
    - Calculate current loss (forward propagation)
    - Calculate current gradient (backward propagation)
    - Update parameters (gradient descent)

You often build 1-3 separately and integrate them into one function we call `model()`.

### 4.1 - Helper functions

**Exercise**: Using your code from "Python Basics", implement `sigmoid()`. As you've seen in the figure above, you need to compute $sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$ to make predictions. Use np.exp().

In [4]:
def sigmoid(z):
    s=1/(1+np.exp(-z))
    return s

In [5]:
sigmoid(np.array([0,2,3,4,5,]))

array([0.5       , 0.88079708, 0.95257413, 0.98201379, 0.99330715])

In [6]:
print('sigmoid of ([0,2,3,4,5,]) is \n=',sigmoid(np.array([0,2,3,4,5,])))

sigmoid of ([0,2,3,4,5,]) is 
= [0.5        0.88079708 0.95257413 0.98201379 0.99330715]


### initilizing parameters

In [7]:
def initialize_with_zeros(dim):
    w=np.zeros((dim,1))
    
    b=0
    

    return w,b

In [8]:
dim=3
w,b=initialize_with_zeros(dim)
print('w = '+ str(w))
print('b = '+ str(b))

w = [[0.]
 [0.]
 [0.]]
b = 0


In [9]:
print(w.shape)

(3, 1)


## forward and backward propogation

Now that your parameters are initialized, you can do the "forward" and "backward" propagation steps for learning the parameters.

**Exercise:** Implement a function `propagate()` that computes the cost function and its gradient.

**Hints**:

Forward Propagation:
- You get X
- You compute $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

Here are the two formulas you will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [10]:
def propogate(w,b,X,Y):
    
    import numpy as np
    
    m=X.shape[1]
    # forward prop
    
    A=sigmoid(np.dot(w.T,X)+b)
    cost=-1./m*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
    
    #back prp
    dw=1./m*np.dot(X,(A-Y).T)
    db=1./m*np.sum((A-Y))
    
    grads={'dw':dw,
           'db':db
        }
    return grads,cost  

In [11]:
import numpy as np
w, b, X, Y = np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]])
grads, cost = propogate(w, b, X, Y)
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))
print ("cost = " + str(cost))

dw = [[0.99845601]
 [2.39507239]]
db = 0.001455578136784208
cost = 5.801545319394553


Expected Output:

        dw 	[[ 0.99845601] [ 2.39507239]]
        db 	0.00145557813678
        cost 	5.801545319394553 



## optimize

Optimization

    You have initialized your parameters.
    You are also able to compute a cost function and its gradient.
    Now, you want to update the parameters using gradient descent.

Exercise: Write down the optimization function. The goal is to learn ww and bb by minimizing the cost function JJ. For a parameter θθ, the update rule is θ=θ−α dθθ=θ−α dθ, where αα is the learning rate.


In [10]:

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
  
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propogate(w, b, X, Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w-learning_rate * dw
        b = b-learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [11]:
params, grads, costs = optimize(w, b, X, Y, num_iterations= 100, learning_rate = 0.009, print_cost = False)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))

w = [[0.19033591]
 [0.12259159]]
b = 1.9253598300845747
dw = [[0.67752042]
 [1.41625495]]
db = 0.21919450454067652


Expected Output:


                    w 	[[ 0.19033591] [ 0.12259159]]
                    b 	1.92535983008
                    dw 	[[ 0.67752042] [ 1.41625495]]
                    db 	0.219194504541 

**Exercise:** The previous function will output the learned w and b. We are able to use w and b to predict the labels for a dataset X. Implement the `predict()` function. There are two steps to computing predictions:

1. Calculate $\hat{Y} = A = \sigma(w^T X + b)$

2. Convert the entries of a into 0 (if activation <= 0.5) or 1 (if activation > 0.5), stores the predictions in a vector `Y_prediction`. If you wish, you can use an `if`/`else` statement in a `for` loop (though there is also a way to vectorize this). 

In [ ]:
def predict(w,b,X):
    m=X.shape[1]
    y_prediction=np.zeros((1,m))
    w=w.reshape(X.shape[0],1)
    
    
    
    A=sigmoid(np.dot(w.T,X)+b)
    
    
    for i in range(A.shape[1]):
        if A[0,i]>0.5:
            y_prediction[0,i]=1
        else:
            y_prediction[0,i]=0
                         
                    
    return y_prediction                      
                         
                             

In [19]:

import numpy as np
w = np.array([[0.1124579],[0.23106775]])
b = -0.3
X = np.array([[1.,-1.1,-3.2],[1.2,2.,0.1]])
print ("predictions = " + str(predict(w, b, X)))

predictions = [[1. 1. 0.]]


## 5 - Merge all functions into a model ##

You will now see how the overall model is structured by putting together all the building blocks (functions implemented in the previous parts) together, in the right order.

**Exercise:** Implement the model function. Use the following notation:
    - Y_prediction_test for your predictions on the test set
    - Y_prediction_train for your predictions on the train set
    - w, costs, grads for the outputs of optimize()

In [21]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    
    # initialize with parameters
    w,b=initialize_with_zeros(x_train.shape[0])
    
    
    # Gadient descent
    
    parameters,grads,costs=optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w=parameters['w']
    b=parameters['b']
    
    #prediction
    y_prediction_test=predict(w,b,X_test)
    y_prediction_train=predict(w,b,X_train)
    
    
    #print train test error
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [3]:
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

ValueError: operands could not be broadcast together with shapes (4,3) (3,2) 

Expected Output:
    
    
    

                        Cost after iteration 0 	0.693147
                        $\vdots$

                        $\vdots$
                        Train Accuracy 	99.04306220095694 %
                        Test Accuracy 	70.0 % 

In [12]:
# Plot learning curve (with costs)
costs = np.squeeze(d['costs'])
plt.plot(costs)
plt.ylabel('cost')
plt.xlabel('iterations (per hundreds)')
plt.title("Learning rate =" + str(d["learning_rate"]))
plt.show()

(4, 1)

In [ ]:
learning_rates = [0.01, 0.001, 0.0001]
models = {}
for i in learning_rates:
    print ("learning rate is: " + str(i))
    models[str(i)] = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 1500, learning_rate = i, print_cost = False)
    print ('\n' + "-------------------------------------------------------" + '\n')

for i in learning_rates:
    plt.plot(np.squeeze(models[str(i)]["costs"]), label= str(models[str(i)]["learning_rate"]))

plt.ylabel('cost')
plt.xlabel('iterations (hundreds)')

legend = plt.legend(loc='upper center', shadow=True)
frame = legend.get_frame()
frame.set_facecolor('0.90')
plt.show()